In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<style>
input {
    background-color: #3c83bb;
    color: white;
    border: none;
    font-family: sans-serif;
    padding: 8px;
    border-radius: 10px;
    
}
</style>
<form action="javascript:code_toggle()"><input type="submit" value="See Raw Code"></form>''')

In [2]:
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive_output
from IPython.display import display, clear_output
from jupyterthemes import jtplot

from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, FactorRange
from bokeh.core.properties import value
from bokeh.layouts import row, column
from bokeh.plotting import figure, show
from bokeh.palettes import d3

jtplot.style(theme='chesterish', grid=False)
output_notebook()


Loading BokehJS ...

In [3]:
df_prod_factors = pd.read_csv("../Data/Productivity_Factors.csv", index_col=0)
df_prod_grades_alpha = pd.read_csv("../Data/Productivity_Grades_Alpha.csv", index_col=0)
df_prod_grades_num = pd.read_csv("../Data/Productivity_Grades_Num.csv", index_col=0)
df_prof_factors = pd.read_csv("../Data/Profitability_Factors.csv", index_col=0)
df_prof_grades_alpha = pd.read_csv("../Data/Profitability_Grades_Alpha.csv", index_col=0, dtype={'High GMR': object, 'High Sales and GM Weight': object, 'Equal Weights': object})
df_prof_grades_num = pd.read_csv("../Data/Profitability_Grades_Num.csv", index_col=0)


In [4]:
df_display_prod = df_prod_grades_num[['Sales District Name', 'Sales Office', 'Date (YYYYMM)', 'High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()
df_display_prod['Year'] = df_display_prod['Date (YYYYMM)'].astype(str).str[0:4].astype(int)
df_display_prod['Month'] = df_display_prod['Date (YYYYMM)'].astype(str).str[4:].astype(int)
    
df_display_prof = df_prof_grades_num[['Sales District Name', 'Sales Office', 'Date (YYYYMM)', 'High GMR', 'High Sales and GM Weight', 'Equal Weights']].copy()
df_display_prof['Year'] = df_display_prof['Date (YYYYMM)'].astype(str).str[0:4].astype(int)
df_display_prof['Month'] = df_display_prof['Date (YYYYMM)'].astype(str).str[4:].astype(int)

timeline = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', 
         '201708', '201709', '201710', '201711', '201712', '201801', '201802', 
         '201803', '201804', '201805', '201806', '201807', '201808', '201809', 
         '201810', '201811', '201812', '201901', '201902', '201903']

month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

branches = df_display_prod['Sales Office'].unique()
district = df_display_prod['Sales District Name'].unique()

district_branch = dict(zip(df_display_prod['Sales Office'], df_display_prod['Sales District Name']))

In [5]:
Productivity = widgets.Dropdown(
    options=['Equal Weights', 'High Sales Weight', 'Equal Sales and GM Weight'],
    value='Equal Weights',
    description='Productivity',
)

Profitability = widgets.Dropdown(
    options=['Equal Weights', 'High GMR', 'High Sales and GM Weight'],
    value='Equal Weights',
    description='Profitability',
)

Fiscal_Year = widgets.Dropdown(
    options=['2017', '2018', '2019'],
    value='2017',
    description='Timeline',
)

District = widgets.Dropdown(
   options=district,
   value='New York',
   description='District',
)

Branch = widgets.Dropdown(
       options=['ALNY','ALPA','HAPA','HVNY','NCDE','PHPA','TENJ','WIPA'],
       value='ALNY',
       description='Branch',
)

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

hbox_1 = widgets.HBox([Productivity, Profitability, Fiscal_Year])
hbox_2 = widgets.HBox([District, Branch])
vbox = widgets.VBox([hbox_1, hbox_2], layout=box_layout)

def update(*args):
    filtered_list=[]
    for k,v in district_branch.items():
        if v==District.value:
            filtered_list.append(k)
    Branch.options=filtered_list
    
def show_this(Productivity, Profitability, Fiscal_Year, District, Branch):
    temp = pd.DataFrame()
    temp_df_display_prod = df_display_prod[(df_display_prod['Year'] == int(Fiscal_Year)) & (df_display_prod['Sales Office'] == Branch)]
    temp_df_display_prof = df_display_prof[(df_display_prof['Year'] == int(Fiscal_Year)) & (df_display_prof['Sales Office'] == Branch)]
    quantities = ['Productivity', 'Profitability']
    if Fiscal_Year!='2019':
        factors = [
        ("Q1", "Jan"), ("Q1", "Feb"), ("Q1", "Mar"),
        ("Q2", "Apr"), ("Q2", "May"), ("Q2", "Jun"),
        ("Q3", "Jul"), ("Q3", "Aug"), ("Q3", "Sep"),
        ("Q4", "Oct"), ("Q4", "Nov"), ("Q4", "Dec"),] 
    else :
        factors = [
        ("Q1", "Jan"), ("Q1", "Feb"), ("Q1", "Mar")]
    source = ColumnDataSource(data = {'x' : factors,
                                      'Branches' : temp_df_display_prod['Sales Office'], 'Year' :temp_df_display_prod['Year'], 'Month' :temp_df_display_prod['Month'],
                                      'Productivity': temp_df_display_prod[Productivity],
                                      'Profitability': temp_df_display_prof[Profitability],
                                      })
    p1 = figure(x_range=FactorRange(*factors), plot_width =450, plot_height=350,toolbar_location=None, tools="hover", tooltips = [('Productivity Score','@Productivity{2.f}'), ('Profitability Score','@Profitability{2.f}')]) 
    p1.vbar_stack(quantities, x='x', width=0.8, alpha=0.5, color=['#00384e', "#3399cc"], source=source ,legend=[value(x) for x in quantities])
    p1.y_range.start = 0
    p1.y_range.end = 80
    p1.x_range.range_padding = 0.005
    p1.xaxis.major_label_orientation = 1
    p1.xgrid.grid_line_color = None
    p1.legend.location = "top_left"
    p1.legend.orientation = "horizontal"
    
    p2 = figure(x_range=FactorRange(*factors), plot_width =450, plot_height=350) 
    p2.y_range.start = 0
    p2.y_range.end = 40
    p2.x_range.range_padding = 0.005
    p2.xaxis.major_label_orientation = 1
    p2.xgrid.grid_line_color = None
    sp1 = p2.line('x' ,'Productivity', source = source, color='#00384e', line_width=1, legend = 'Productivity Trend Line')
    sp2 = p2.line('x' , 'Profitability', source = source, color="#3399cc", line_width=1, legend = 'Profitability Trend Line')
    p2.add_tools(HoverTool(renderers = [sp1], tooltips = [('Productivity Score', '@Productivity{2.f}'),('Month', '@Month')]))
    p2.add_tools(HoverTool(renderers = [sp2], tooltips = [('Profitability Score', '@Profitability{2.f}'),('Month', '@Month')]))
    p2.legend.location = "bottom_right"
    show(row(p1,p2))

District.observe(update)    
box = interactive_output(show_this, {'Productivity':Productivity, 'Profitability':Profitability, 'Fiscal_Year':Fiscal_Year, 'District':District, 'Branch':Branch})

In [6]:
display(box, vbox)

Output()

<center>
    <img src=../Legend/Productivity_and_Profitability_Graphs.png  width="500000" height="250000" />
</center